In [1]:
pip install tweetpy

In [2]:
pip install s3fs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


In [9]:
import tweepy
import pandas as pd
import json
from datetime import datetime
import s3fs

def run_twitter_etl():
  api_key = 'KllHvsNElb26nELjt1mB5gDpz'
  API_Key_Secret = 'ztJKeGUmEPoi1cbXjNSdBb3LW9ial9kWTTpOuxrZK9SUjGPiUy'
  access_token = '1411140158133395456-DCEyPPf8EZDR1uhivCVKMNUOalqiUz'
  access_token_secret = 'G4WsmibATLxPj5eNAqbN22YN8sNYahKAV3PCPYf0AhV8W'

  auth = tweepy.OAuthHandler(api_key, API_Key_Secret)
  auth.set_access_token(access_token, access_token_secret)


  api = tweepy.API(auth)

  tweets = api.user_timeline(screen_name = '@elonmusk',
                            count = 2,
                            include_rts = False,
                            tweet_mode = 'extended')

  tweets_list= []
  for tweet in tweets:
    text = tweet._json['full_text']

    refined_tweet = {
        'user': tweet.user.screen_name,
        'text':text,
        'favorite count': tweet.favorite_count,
        'retweet_count': tweet.retweet_count,
        'created_at': tweet.created_at
    }
    tweets_list.append(refined_tweet)

  df = pd.DataFeame(tweets_list)
  df.to_csv('airflow_1225.csv')

Forbidden: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product

In [ ]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python operator import PythonOperator
from airflow.utils.dates import days_ago
from airflow.utils.dates import days_ago
from datetime import timedelta

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime[2024,12,25],
    'email': ['ddd@gmail.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=1)
}

dag = DAG(
    'twitter_dag',
    default_args = default_args,
    description='My First erl code'
)

run_etl = PythonOperator(
    task_id='complete_twitter_etl',
    python_callable=run_twitter_etl,
    dag=dag,
)

run_etl